In [1]:
from bs4 import BeautifulSoup
from google.colab import drive
import requests
import pandas as pd
import time

In [10]:
##SET UP
extract_all = True
extract_specific_years = False
years_to_extract = range(2004, 2025)

In [2]:
base_url = 'https://www.motorcycle.com'
response = requests.get(f'{base_url}/specs/')
soup = BeautifulSoup(response.text, 'html.parser')

In [14]:
brands = soup.find('ul',class_='brands-list order minus-one sl-all-brands-list').find_all('a')
brands = [[names, links] for names, links in zip([name.text.strip() for name in brands], [link['href'] for link in brands])]
brands = pd.DataFrame(brands, columns=['name', 'link'])
brands_to_scrap = ['Aprilia', 'BMW', 'Ducati', 'Kawasaki', 'KTM', 'Triumph', 'CFMoto', 'Honda', 'Indian', 'Yamaha', 'Harley-Davidson', 'Suzuki', 'Benelli', 'Buell', 'Erik Buell Racing', 'GAS GAS', 'Husqvarna', 'Kymco', 'LiveWire', 'Moto Guzzi', 'MV Agusta', 'Piaggio', 'Royal Enfield', 'SSR Motorsports', 'Vespa', 'Zero']
brands = brands[brands['name'].isin(brands_to_scrap)]

In [ ]:
# Looping thru the brands to get all of them
if extract_all:
  moto_dataset_all = pd.DataFrame()
  for index, row in brands.iterrows():
    moto_dataset = pd.DataFrame()

    brand_name = row['name']
    brand_url = row['link']
    print(brand_name)

    brand_response = requests.get(f'{base_url}{brand_url}')
    brand_soup = BeautifulSoup(brand_response.text, 'html.parser')

    if brand_soup.find('div', class_='pagination-wrap') is not None:
      total_pages = int(brand_soup.find('div', class_='pagination-wrap')['data-total-pages'])+1
      total_pages = range(1,total_pages)
    else:
      total_pages = range(1,2)

    # Looping thru the pages inside the brand to get all the motorcycles
    for page in total_pages:
      time.sleep(1)
      print("="*5, page)

      page_response = requests.get(f'{base_url}{brand_url}?page_num={page}')
      page_soup = BeautifulSoup(page_response.text, 'html.parser')

      card_content = page_soup.find('div', class_='spec-page-feed').find_all('div', 'feed-spec-card')
      moto_list_url = [moto_card.find('a', class_='card-link')['href'] for moto_card in card_content]

      # Looping thru every motorcycle whitin a page to get all its stats
      for moto_url in moto_list_url:

        print("="*10, moto_url)
        moto_response = requests.get(f'{base_url}{moto_url}')
        moto_soup = BeautifulSoup(moto_response.text, 'html.parser')######

        moto_img = moto_soup.find('img', class_='gallery-image sl-gallery-image')['src']
        base_specs = {'Brand name': brand_name,
                      'Moto image': moto_img,
                      'profile link': f'{base_url}{moto_url}'
                      }

        all_specs_raw = moto_soup.find_all('div', class_='vs-specs-table-row')
        all_specs = dict((specs.find('div', 'spec-key bold').text.strip(), specs.find('div', 'spec-value').text.strip()) for specs in all_specs_raw)
        all_specs.update(base_specs)

        moto_df = pd.DataFrame(all_specs, index=[0])
        moto_dataset = pd.concat([moto_dataset, moto_df], ignore_index=True)
        moto_dataset_all = pd.concat([moto_dataset_all, moto_df], ignore_index=True)
        time.sleep(5)

    moto_dataset.to_json(f'/content/drive/My Drive/data scrap/motopy/JSON/{brand_name}.json', orient='records', lines=True)

In [ ]:
# Honda its broken in the website so I had to do something extra for it, and scrap year by year
if extract_specific_years:
  moto_dataset_all = pd.DataFrame()

  for index, row in brands.iterrows():
    brand_name = row['name']
    brand_url = row['link']
    print(brand_name)
    years_missing = list(years_to_extract)

    for year in years_missing:
      moto_dataset = pd.DataFrame()

      link = f'{base_url}/specs/{brand_name}/{year}.html'
      brand_url = '/'.join(link.split('/')[:-1])

      brand_response = requests.get(link)
      brand_soup = BeautifulSoup(brand_response.text, 'html.parser')

      if brand_soup.find('div', class_='pagination-wrap') is not None:
        total_pages = int(brand_soup.find('div', class_='pagination-wrap')['data-total-pages'])+1
        total_pages = range(1,total_pages)
      else:
        total_pages = range(1,2)

      # Looping thru the pages inside the brand to get all the motorcycles
      for page in total_pages:
        time.sleep(1)
        print("="*5, page)

        page_response = requests.get(f'{link}?page_num={page}')
        page_soup = BeautifulSoup(page_response.text, 'html.parser')

        card_content = page_soup.find('div', class_='spec-page-feed').find_all('div', 'feed-spec-card')
        moto_list_url = [moto_card.find('a', class_='card-link')['href'] for moto_card in card_content]

        # Looping thru every motorcycle whitin a page to get all its stats
        for moto_url in moto_list_url:

          print("="*10, moto_url)
          moto_response = requests.get(f'{base_url}{moto_url}')
          moto_soup = BeautifulSoup(moto_response.text, 'html.parser')

          moto_img = moto_soup.find('img', class_='gallery-image sl-gallery-image')['src']
          base_specs = {'Brand name': brand_name,
                        'Moto image': moto_img,
                        'profile link': f'{base_url}{moto_url}'}

          all_specs_raw = moto_soup.find_all('div', class_='vs-specs-table-row')
          all_specs = dict((specs.find('div', 'spec-key bold').text.strip(), specs.find('div', 'spec-value').text.strip()) for specs in all_specs_raw)
          all_specs.update(base_specs)

          moto_df = pd.DataFrame(all_specs, index=[0])
          moto_dataset = pd.concat([moto_dataset, moto_df], ignore_index=True)
          moto_dataset_all = pd.concat([moto_dataset_all, moto_df], ignore_index=True)
          time.sleep(2)

  moto_dataset_all.to_json(f'/content/drive/My Drive/data scrap/motopy/JSON/{brand_name}-todelete.json', orient='records', lines=True)